In [1]:
#Read in the example df
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install transformers

In [3]:
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm
from transformers import AutoTokenizer, AutoModel


In [4]:
TOKENIZER =  AutoTokenizer.from_pretrained(
        'nlpaueb/bert-base-greek-uncased-v1')
tokenizer = TOKENIZER

In [5]:
cd 'bert2/squad2.0/'

/content/drive/My Drive/bert2/squad2.0


In [6]:
import numpy as np
import torch
import pandas as pd
import os


class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score


def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [7]:
cd squad2.0/

[Errno 2] No such file or directory: 'squad2.0/'
/content/drive/My Drive/bert2/squad2.0


In [8]:
df1 = pd.read_excel('result_df_with_sentence_20000.xlsx')
del df1['Unnamed: 0']
del df1['Ανώνυμη: 0']

# fix περιβαλλον
df1['περιβάλλον'] = df1.περιβάλλον.fillna(method='ffill')
df1.shape

(20000, 8)

In [9]:
##### Strip sentence
strip_sent_list= []
start_str_index_list = []
end_str_index_list = []

strt = df1.λίστα_έναρξης_ποσοστού*df1.περιβάλλον.str.len()
start_str_index = strt.astype(int) - 40

nd = df1.τελικό_ποσοστό*df1.περιβάλλον.str.len()
end_str_index = nd.astype(int) + 45

for index, row in df1.iterrows():
    if start_str_index[index]<0:
      safe_start = 0
    else:
      safe_start = start_str_index[index]
    strip_sent = row.περιβάλλον[safe_start:end_str_index[index]]
    strip_sent_list.append(' '.join(strip_sent.split(' ')[1:-1]))
    start_str_index_list.append(start_str_index[index])
    end_str_index_list.append(end_str_index[index])
    if len(strip_sent)==0:
      print("\n")
      print(f"row.περιβάλλον :{row.περιβάλλον}")
      print(f"row.κείμενο :{row.κείμενο}")
      print(f"row.λίστα_έναρξης_ποσοστού :{row.λίστα_έναρξης_ποσοστού}")
      print(f"row.τελικό_ποσοστό :{row.τελικό_ποσοστό}")
      print(f"start_str_index :{start_str_index[index]}")
      print(f"end_str_index :{end_str_index[index]}")
      print(f"strip_sent :{strip_sent}")
df1['striped'] = strip_sent_list
df1['start_str_index_list'] = start_str_index_list
df1['end_str_index_list'] = end_str_index_list

In [10]:

df2 = pd.read_csv('csv_correct2_20000_40000.csv')
del df2['Unnamed: 0']

# fix περιβαλλον
df2['περιβάλλον'] = df2.περιβάλλον.fillna(method='ffill')
df2 = df2.iloc[1:]

In [11]:
#df2 = pd.read_excel('df_20000_40000_correct.xlsx')
df3 = pd.read_csv('csv_correct2_60000_80000.csv')
del df3['Unnamed: 0']

# fix περιβαλλον
df3['περιβάλλον'] = df3.περιβάλλον.fillna(method='ffill')
df3 = df3.iloc[1:]

In [12]:
# Merge
df = df1.append(df2, ignore_index = True)

In [13]:
# Merge
df = df.append(df3, ignore_index = True)

In [14]:
df.shape

(59993, 12)

In [15]:
#filter if size > 512
for index, row in df.iterrows():
    #print("row['περιβάλλον'] : {}".format(row['περιβάλλον']))
    tok_tweet = tokenizer.encode_plus(row['περιβάλλον'],add_special_tokens=True,return_offsets_mapping=True)
    input_ids_orig = tok_tweet.input_ids
    if len(input_ids_orig)>422:
        df.drop(index, inplace=True)

In [16]:
df.shape

(59445, 12)

In [17]:
df2.head(1)

,ερώτηση,περιβάλλον,κείμενο,πρόταση,έναρξη_απάντησης,αναγνωριστικό_c,εκκίνηση_percen,τελικό_ποσοστό,striped,start_str_index_list,end_str_index_list
1,"Μαζί με τον Άγιο Παύλο, ποια πόλη αποτελεί τις...",Περίπου 20.000 Σομαλοί μετανάστευσαν στην πολι...,Μινεάπολη,Η πολιτεία της Μινεσότα πριν από περίπου δέκα...,109,4732,0.184433,0.203046,περίπου δέκα χρόνια και οι Δίδυμες Πόλεις (Μιν...,80,177


In [18]:
# if κειμενο is in the striped (case - transparent)

for index, row in df.iterrows():
  try:
    if row["κείμενο"].lower() in row["striped"].lower():
      pass
    else:
      df.drop(index, inplace = True)
  except:
       df.drop(index, inplace = True)

In [19]:
df = df[df.striped == df.striped]

In [20]:
df.shape

(21757, 12)

In [21]:
for index, row in df.head(1).iterrows():
    print(f"περιβάλλον : {row['περιβάλλον']}")
    print(f"ερώτηση : {row['ερώτηση']}")
    print(f"απαντηση : {row['κείμενο']}")  
    print(f"striped : {row['striped']}") 


περιβάλλον : Η Beyoncé Giselle Knowles-Carter (/ːɒ nseɪ/ bee-YON-say) (γεννημένη στις 4 Σεπτεμβρίου 1981) είναι Αμερικανίδα τραγουδίστρια, συνθέτης, παραγωγός δίσκων και ηθοποιός. Γεννημένη και μεγαλωμένη στο Χιούστον του Τέξας, συμμετείχε σε διάφορους διαγωνισμούς τραγουδιού και χορού ως παιδί και έγινε διάσημη στα τέλη της δεκαετίας του 1990 ως τραγουδίστρια της R&B girl-group Destiny's Child. Υπό τη διαχείριση του πατέρα της, Μάθιου Νόουλς, η ομάδα έγινε μια από τις πιο γνωστές παγκοσμίως ομάδες κοριτσιών. Η χασμουρητό τους είδε την κυκλοφορία του πρώτου άλμπουμ της Beyoncé, Dangerously in Love (2003), το οποίο την καθιέρωσε ως σόλο καλλιτέχνη παγκοσμίως, κέρδισε πέντε βραβεία Grammy και χαρακτήρισε το Billboard Hot 100 νούμερο ένα singles "Crazy in Love" και "Baby Boy".
ερώτηση : Πότε η Beyonce άρχισε να γίνεται δημοφιλής;
απαντηση : τέλη της δεκαετίας του 1990
striped : και χορού ως παιδί και έγινε διάσημη στα τέλη της δεκαετίας του 1990 ως τραγουδίστρια της R&B


In [22]:
for index, row in df[111:112].iterrows():
    print(f"περιβάλλον : {row['περιβάλλον']}")
    print(f"ερώτηση : {row['ερώτηση']}")
    print(f"απαντηση : {row['κείμενο']}")  
    print(f"striped : {row['striped']}") 

περιβάλλον : Η Beyoncé και ο σύζυγος Jay Z είναι φίλοι με τον Πρόεδρο Μπαράκ Ομπάμα και την Πρώτη Κυρία Μισέλ Ομπάμα. Πραγματοποίησε το "America the Beautiful" στα προεδρικά εγκαίνια του 2009, καθώς επίσης το "At Last" κατά τον πρώτο εναρκτήριο χορό στο Χορό της Γειτονιάς δυο μέρες αργότερα. Η Beyoncé και ο Jay Z οργάνωσαν έρανο στο κλαμπ 40/40 στο Μανχάταν για την προεδρική εκστρατεία του Ομπάμα το 2012, η οποία συγκέντρωσε 4 εκατομμύρια δολάρια. Η Beyoncé δημοσίευσε φωτογραφίες της χάρτινης ψηφοφορίας της στο Tumblr, επιβεβαιώνοντας ότι είχε ψηφίσει υπέρ του Δημοκρατικού Κόμματος και ενθάρρυνε και άλλους να το πράξουν. Επίσης, κατά τη δεύτερη εγκατάστασή του, ερμήνευσε τον αμερικανικό εθνικό ύμνο, τραγουδώντας μαζί με ένα ηχογραφημένο κομμάτι. Υποστήριξε δημόσια το γάμο ατόμων του ιδίου φύλου στις 26 Μαρτίου 2013, μετά τη συζήτηση του Ανωτάτου Δικαστηρίου για την Πρόταση 8 της Καλιφόρνια. Τον Ιούλιο του 2013, η Beyoncé και ο Jay-Z παρακολούθησαν ένα συλλαλητήριο ως απάντηση στην αθώω

In [23]:
class config:
    MAX_LEN = 512
    TRAIN_BATCH_SIZE = 12
    VALID_BATCH_SIZE = 12
    EPOCHS = 3
    BERT_PATH = "../input/bert-base-uncased/"
    MODEL_PATH = "model.bin"
    TRAINING_FILE = "../input/tweet-train-folds/train_folds.csv"

    TOKENIZER =  AutoTokenizer.from_pretrained(
        'nlpaueb/bert-base-greek-uncased-v1')
    


In [24]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    len_st = len(selected_text)
    idx0 = None
    idx1 = None


    #print("tweet :{}".format(tweet))
    #print("sentiment :{}".format(sentiment))
    #print("selected_text :{}".format(selected_text))
 
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):       
        if tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1

    tok_tweet = tokenizer.encode_plus(tweet,add_special_tokens=True,return_offsets_mapping=True)
    input_ids_orig = tok_tweet.input_ids[1:-1]
    tweet_offsets = tok_tweet.offset_mapping[1:-1]
     
    tok_q = tokenizer.encode_plus(sentiment,add_special_tokens=True,return_offsets_mapping=True)
    q_ids_orig = tok_q.input_ids[1:-1]
    q_offsets = tok_q.offset_mapping[1:-1]
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    #print("target_idx : {}".format(target_idx))
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    
    input_ids = [101] + q_ids_orig + [102] + input_ids_orig + [102]
    token_type_ids = [0] + [0] * len(q_ids_orig) + [0] + [1] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] + [(0, 0)] * len(q_offsets) + [(0, 0)] + tweet_offsets + [(0, 0)]
    targets_start += 1 +  len(q_offsets) + 1
    targets_end += 1 +  len(q_offsets) + 1


    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)

    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

In [25]:
class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = AutoTokenizer.from_pretrained(
        'nlpaueb/bert-base-greek-uncased-v1')
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )



        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [26]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.bert = transformers.BertModel.from_pretrained('nlpaueb/bert-base-greek-uncased-v1', config=conf)
    
        for name, param in self.bert.named_parameters():
            if '9' in name:
              param.requires_grad = True
            else:
              #pass
              param.requires_grad = False
            if '10' in name:
              param.requires_grad = True
            if '11' in name:
              param.requires_grad = True
            if '8' in name:
              param.requires_grad = True
            if '7' in name:
              param.requires_grad = True
            if '6' in name:
              param.requires_grad = True
            if '5' in name:
              param.requires_grad = True
            if '10' in name:
              param.requires_grad = True
            if '11' in name:
              param.requires_grad = True
            if 'pooler.dense' in name:
              param.requires_grad = True

        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, attention_mask, token_type_ids):
        #print("in tweet model : type(mask): {}".format({type(attention_mask)}))
        outputs = self.bert(
            ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        #print("in tweet model : out: {}".format({out}))
        out = torch.cat((outputs.hidden_states[-1], 
        outputs.hidden_states[-2]), dim=-1)

        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [27]:
bert = transformers.BertModel.from_pretrained('nlpaueb/bert-base-greek-uncased-v1')


Some weights of the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

In [29]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses = AverageMeter()
    jaccards = AverageMeter()

    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for bi, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        model.zero_grad()
        outputs_start, outputs_end = model(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_start_observe = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end_observe = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        if bi % 5000 == 0 :
          idx_start=np.argmax(outputs_start_observe[0, :])
          idx_end=np.argmax(outputs_end_observe[0, :])
          print("idx_start : {}".format(idx_start))
          print("idx_end : {}".format(idx_end))
          idx_start=np.argmax(outputs_start_observe[1, :])
          idx_end=np.argmax(outputs_end_observe[1, :])
          print("idx_start : {}".format(idx_start))
          print("idx_end : {}".format(idx_end))
          idx_start=np.argmax(outputs_start_observe[2, :])
          idx_end=np.argmax(outputs_end_observe[2, :])
          print("idx_start : {}".format(idx_start))
          print("idx_end : {}".format(idx_end))
          idx_start=np.argmax(outputs_start_observe[3, :])
          idx_end=np.argmax(outputs_end_observe[3, :])
          print("idx_start : {}".format(idx_start))
          print("idx_end : {}".format(idx_end))
          print("targets_start : {}".format(targets_start))
          print("targets_end : {}".format(targets_end))  

        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)      
        loss.backward()
        optimizer.step()
        scheduler.step()
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

        jaccard_scores = []
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            jaccard_score, _ = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            jaccard_scores.append(jaccard_score)

        jaccards.update(np.mean(jaccard_scores), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)

In [30]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output


def eval_fn(data_loader, model, device):
    model.eval()
    losses = AverageMeter()
    jaccards = AverageMeter()
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            outputs_start, outputs_end = model(
                ids,
                attention_mask=mask,
                token_type_ids=token_type_ids
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)
    
    print(f"Jaccard = {jaccards.avg}")
    return jaccards.avg

In [31]:
#NaN
df = df[df.striped == df.striped]

In [32]:
#All lowercase
import unicodedata

def strip_accents_and_lowercase(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').lower()


for index, row in df.iterrows():
#for index, row in train.iterrows():
    df.at[index, 'περιβάλλον'] = strip_accents_and_lowercase(row['περιβάλλον'])
    df.at[index, 'ερώτηση'] = strip_accents_and_lowercase(row['ερώτηση'])
    
    df.at[index, 'κείμενο'] = strip_accents_and_lowercase(row['κείμενο'])
    df.at[index, 'striped'] = strip_accents_and_lowercase(row['striped'])
    df.at[index, 'πρόταση'] = strip_accents_and_lowercase(row['πρόταση'])

In [33]:
def run(fold):

    df_train = df[:18000].reset_index(drop=True)
    df_valid = df[18000:].reset_index(drop=True)
    
    train_dataset = TweetDataset(
        tweet=df_train.περιβάλλον.values,
        sentiment=df_train.ερώτηση.values,
        selected_text=df_train.κείμενο.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=1
    )

    valid_dataset = TweetDataset(
    tweet=df_valid.περιβάλλον.values,
    sentiment=df_valid.ερώτηση.values,
    selected_text=df_valid.κείμενο.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=1
    )

    device = torch.device("cuda")
    model_config = transformers.BertConfig.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
    model_config.output_hidden_states = True
    model = transformers.BertModel.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")#
    model = TweetModel(conf=model_config)
    model.to(device)

    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    es = EarlyStopping(patience=3, mode="max")
    print(f"Training is Starting for fold={fold}")
    
    # I'm training only for 3 epochs even though I specified 5!!!
    for epoch in range(4):
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        jaccard = eval_fn(valid_data_loader, model, device)
        print(f"Jaccard Score = {jaccard}")
        es(jaccard, model, model_path=f"model_5_7{fold}.bin")
        torch.save(model, "model_4_8_ALL{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break

In [34]:
run(fold=0)

Some weights of the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 were not used when initializing

Training is Starting for fold=0


  0%|          | 0/1500 [00:00<?, ?it/s]

idx_start : 432
idx_end : 11
idx_start : 463
idx_end : 502
idx_start : 67
idx_end : 499
idx_start : 195
idx_end : 369
targets_start : tensor([ 82, 110, 171, 180,  91, 189, 140,  88,  24, 105, 115, 113],
       device='cuda:0')
targets_end : tensor([ 86, 113, 171, 180,  93, 189, 143,  90,  28, 106, 116, 114],
       device='cuda:0')


  0%|          | 0/314 [00:00<?, ?it/s]

...

Jaccard = 0.6487291895050082
Jaccard Score = 0.6487291895050082
Validation score improved (0.6386719489248687 --> 0.6487291895050082). Saving model!


  0%|          | 0/1500 [00:00<?, ?it/s]

idx_start : 82
idx_end : 86
idx_start : 110
idx_end : 113
idx_start : 171
idx_end : 171
idx_start : 180
idx_end : 180
targets_start : tensor([ 82, 110, 171, 180,  91, 189, 140,  88,  24, 105, 115, 113],
       device='cuda:0')
targets_end : tensor([ 86, 113, 171, 180,  93, 189, 143,  90,  28, 106, 116, 114],
       device='cuda:0')


  0%|          | 0/314 [00:00<?, ?it/s]

Jaccard = 0.6487291895050082
Jaccard Score = 0.6487291895050082
EarlyStopping counter: 1 out of 3


Inference

In [ ]:
model_config = transformers.BertConfig.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
model_config.output_hidden_states = True
device = torch.device("cuda")
#no
model1 = TweetModel(conf=model_config)
model1.to(device)
#model1 = nn.DataParallel(model1)
model1.load_state_dict(torch.load("./model_5_70.bin"), strict=False)
model1.eval()

In [36]:
final_output = []

# Pick a random example
df_valid = df[19522:19523]

valid_dataset = TweetDataset(
tweet=df_valid.περιβάλλον.values,
sentiment=df_valid.ερώτηση.values,
selected_text=df_valid.κείμενο.values
)

data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=config.VALID_BATCH_SIZE,
    num_workers=1
)


with torch.no_grad():
    tk0 = tqdm(data_loader, total=len(data_loader))
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"].numpy()

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        outputs_start, outputs_end = model1(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        

        jaccard_scores = []
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            _, output_sentence = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            final_output.append(output_sentence)


for index, row in df[19522:19523].head(1).iterrows():
    print(f"περιβάλλον : {row['περιβάλλον']}")
    print(f"ερώτηση : {row['ερώτηση']}")
    print(f"απαντηση : {row['κείμενο']}")
    print(f"striped : {row['striped']}")
    print(f"final_output : {final_output}")

  0%|          | 0/1 [00:00<?, ?it/s]

περιβάλλον : περιλαμβανει τοσο κοσμικη και θρησκευτικη στιλ, το σχεδιο και το στιλ που γινεται απο τους μουσουλμανους και την κατασκευη τους κτιρια και τις δομες στην ισλαμικη κουλτουρα περιλαμβανουν τους αρχιτεκτονικους τυπους: το τζαμι, ο ταφος, το παλατι και το φρουριο. ισως η πιο σημαντικη εκφραση της ισλαμικης τεχνης ειναι η αρχιτεκτονικη, ιδιαιτερα αυτη του τζαμιου. μεσω της ισλαμικης αρχιτεκτονικης, μπορουν να απεικονιστουν οι επιπτωσεις διαφορετικων πολιτισμων μεσα στον ισλαμικο πολιτισμο. γενικα, η χρηση των ισλαμικων γεωμετρικων μοτιβων και των λαικων αραβικων ηταν εντυπωσιακη. υπηρξε επισης η χρηση διακοσμητικης καλλιγραφιας αντι για εικονες που ηταν χαραμ (απαγορευμενες) στην αρχιτεκτονικη τζαμιου. σημειωστε οτι στην κοσμικη αρχιτεκτονικη υπηρχε πραγματι αναπαρασταση ανθρωπων και ζωων.
ερώτηση : που στη μουσουλμανικη αρχιτεκτονικη απαγορευονται οι εικονες;
απαντηση : αρχιτεκτονικη τζαμιου
striped : που ηταν χαραμ (απαγορευμενες) στην αρχιτεκτονικη τζαμιου. σημειωστε οτι στη